# Working with Daily Data in Cal-Adapt API

In [1]:
import requests 
import json
headers = {'ContentType': 'json'}

api = 'http://api.cal-adapt.org/api'
resource = 'series'

### Get a list of daily timeseries from Cal-Adapt API

In [2]:
search_str = 'daily+maximum+temperature'
params = {'name': search_str, 'pagesize': 100}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = api + '/' + resource + '/' + '?' + params_str
url

'http://api.cal-adapt.org/api/series/?name=daily+maximum+temperature&pagesize=100'

In [3]:
timeseries = None
response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    data = response.json()
    timeseries = data['results']
    for item in timeseries:
        print(item['name'])

daily maximum temperature ACCESS1-0 modeled historical
daily maximum temperature ACCESS1-0 RCP 4.5
daily maximum temperature ACCESS1-0 RCP 8.5
daily maximum temperature CCSM4 modeled historical
daily maximum temperature CCSM4 RCP 4.5
daily maximum temperature CCSM4 RCP 8.5
daily maximum temperature CESM1-BGC modeled historical
daily maximum temperature CESM1-BGC RCP 4.5
daily maximum temperature CESM1-BGC RCP 8.5
daily maximum temperature CMCC-CMS modeled historical
daily maximum temperature CMCC-CMS RCP 4.5
daily maximum temperature CMCC-CMS RCP 8.5
daily maximum temperature CNRM-CM5 modeled historical
daily maximum temperature CNRM-CM5 RCP 4.5
daily maximum temperature CNRM-CM5 RCP 8.5
daily maximum temperature CanESM2 modeled historical
daily maximum temperature CanESM2 RCP 4.5
daily maximum temperature CanESM2 RCP 8.5
daily maximum temperature GFDL-CM3 modeled historical
daily maximum temperature GFDL-CM3 RCP 4.5
daily maximum temperature GFDL-CM3 RCP 8.5
daily maximum temperature 

### Request daily data for a timeseries and location

In [4]:
lat = 38.5938
lng = -121.4687
point = 'POINT(' + str(lng) + ' ' + str(lat) + ')' #WKT format
params = {'g': point}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())

ts = [x for x in timeseries if (x['name'].find('Livneh') > -1)]
observed_ts = ts[0]
observed_ts

{'begin': '1950-01-01T00:00:00Z',
 'end': '2013-12-31T00:00:00Z',
 'name': 'daily maximum temperature Livneh',
 'rasters': ['http://api.cal-adapt.org/api/rstores/tasmax_day_livneh_1950-2013/'],
 'slug': 'tasmax_day_livneh',
 'tags': ['climate', 'tasmax', 'temperature'],
 'url': 'http://api.cal-adapt.org/api/series/tasmax_day_livneh/'}

In [5]:
from datetime import datetime
from itertools import chain
import pandas as pd

data = None

url = observed_ts['url'] + 'rasters/?' + params_str
response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    json = response.json()
    data = json['results'][0]
    # Multiband raster data is returned as a 3 dimensional array
    print(data)

{'id': 14643, 'tileurl': 'http://api.cal-adapt.org/tiles/tasmax_day_livneh_1950-2013/{z}/{x}/{y}.png', 'url': 'http://api.cal-adapt.org/api/rstores/tasmax_day_livneh_1950-2013/', 'image': [[[6.929999828338623]], [[4.829999923706055]], [[8.850000381469727]], [[6.619999885559082]], [[7.309999942779541]], [[9.569999694824219]], [[10.539999961853027]], [[9.229999542236328]], [[10.640000343322754]], [[9.520000457763672]], [[8.8100004196167]], [[10.880000114440918]], [[6.619999885559082]], [[6.860000133514404]], [[9.779999732971191]], [[5.949999809265137]], [[11.069999694824219]], [[14.65999984741211]], [[16.979999542236328]], [[18.850000381469727]], [[15.84000015258789]], [[18.389999389648438]], [[17.889999389648438]], [[15.210000038146973]], [[10.710000038146973]], [[7.429999828338623]], [[9.989999771118164]], [[7.690000057220459]], [[9.6899995803833]], [[10.430000305175781]], [[14.520000457763672]], [[9.4399995803833]], [[10.579999923706055]], [[11.289999961853027]], [[11.170000076293945]

### Create pandas dataframe from daily timeseries

In [6]:
# Get start date of timeseries
start_date = datetime.strptime(data['event'], '%Y-%m-%d')
# Get units for timeseries
units = data['units']
# Get values - 3D array
values_list = data['image']
# Flatten list
flattened_list = list(chain.from_iterable(values_list))
# Get total number of values -> number of days
length = len(flattened_list)
# Create new pandas dataframe and map each value in list to a date index
df = pd.DataFrame(flattened_list,
    index=pd.date_range(start_date, freq='1D', periods=length),
    columns=['Value'])
print(df.head())
print()
print(df.tail())

            Value
1950-01-01   6.93
1950-01-02   4.83
1950-01-03   8.85
1950-01-04   6.62
1950-01-05   7.31

                Value
2013-12-27  14.750000
2013-12-28  19.549999
2013-12-29  17.680000
2013-12-30  18.570000
2013-12-31  17.799999


### Filter dataframe for years 1961 - 1990 and months April - October (Baseline Period)

In [7]:
filtered = df.ix['1961-01-01':'1990-12-31']
filtered = filtered[(filtered.index.month >= 4) & (filtered.index.month <= 10)]
print(filtered.head())
print()
print(filtered.tail())

                Value
1961-04-01  24.740000
1961-04-02  27.969999
1961-04-03  29.620001
1961-04-04  27.450001
1961-04-05  26.639999

                Value
1990-10-27  28.760000
1990-10-28  27.350000
1990-10-29  24.110001
1990-10-30  22.580000
1990-10-31  21.690001


### Calculate extreme heat threshold

In [8]:
threshold = filtered['Value'].quantile(0.98, interpolation='linear')
thresholdF = (threshold * 9/5) + 32
print('Extreme Heat Threshold for', point, 'is', round(thresholdF, 1), 'degrees Fahrenheit', sep = ' ')

Extreme Heat Threshold for POINT(-121.4687 38.5938) is 103.9 degrees Fahrenheit
